In [ ]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip

--2021-12-01 19:01:36--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.147.128, 142.250.125.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.147.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   135MB/s    in 1.1s    

2021-12-01 19:01:37 (135 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [ ]:
!unzip horse-or-human.zip

In [ ]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2021-12-01 19:01:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.128, 142.251.6.128, 142.250.159.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip’

validation-horse-or 100%[===================>]  10.95M  --.-KB/s    in 0.07s   

2021-12-01 19:01:45 (168 MB/s) - ‘validation-horse-or-human.zip’ saved [11480187/11480187]



In [ ]:
!unzip validation-horse-or-human.zip

In [ ]:
i = 0
for filename in os.listdir("/content/horses"):
  os.rename(f"/content/horses/{filename}", f"/content/horses/{i}.png")
  i+=1

In [ ]:
i = 0
for filename in os.listdir("/content/humans"):
  os.rename(f"/content/humans/{filename}", f"/content/humans/{i}.png")
  i+=1

In [ ]:
import shutil
import os

In [ ]:
!mkdir /content/HuHImages

In [ ]:
!cp -r /content/horses /content/HuHImages

In [ ]:
!cp -r /content/humans /content/HuHImages

In [ ]:
#Каталог с набором данных
data_dir = '/content/HuHImages/'
#Каталог с данными для обучения
train_dir = 'train'
#Каталог с данными для проверки
val_dir   = 'val'
#Каталог с данными для тестирования
test_dir  = 'test'
#Часть набора данных для тестривания
test_portion = 0.15
#Часть набора данных для проверки
val_portion = 0.15
#Количество элементов данных в одном классе
nb_images = 500

In [ ]:
def create_dir(dir_name):
  
  if os.path.exists(dir_name):
    shutil.rmtree(dir_name)

  os.makedirs(dir_name)
  os.makedirs(os.path.join(dir_name, "horses"))
  os.makedirs(os.path.join(dir_name, "humans"))

In [ ]:
create_dir(train_dir)
create_dir(test_dir)
create_dir(val_dir)

In [ ]:
def copy_images(st, ed, src, dst):
  for i in range(st, ed):
    shutil.copy2(os.path.join(src, "horses/" + str(i) + ".png"), os.path.join(dst, "horses"))
    shutil.copy2(os.path.join(src, "humans/" + str(i) + ".png"), os.path.join(dst, "humans"))

In [ ]:
start_val = int(nb_images * (1 - val_portion - test_portion))
start_tst = int(nb_images * (1 - test_portion))
print(start_val)
print(start_tst)
# train - 0..8749
# val   - 8750 .. 10624
# test  - 10625 .. 12500

350
425


In [ ]:
# train
copy_images(0, start_val, data_dir, train_dir)

In [ ]:
copy_images(start_val, start_tst, data_dir, val_dir)

In [ ]:
copy_images(start_tst, nb_images, data_dir, test_dir)

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 1
batch_size = 20
nb_train   = 700
nb_val     = 150
nb_test    = 150

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = datagen.flow_from_directory(train_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 700 images belonging to 2 classes.


In [ ]:
val_generator = datagen.flow_from_directory(val_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 150 images belonging to 2 classes.


In [ ]:
test_generator = datagen.flow_from_directory(test_dir, 
                                              target_size=(img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode='binary')

Found 150 images belonging to 2 classes.


In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
inception_v3_net = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(150, 150, 3))
inception_v3_net.trainable=False

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Activation, Dense
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()
model.add(inception_v3_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_4 (Flatten)         (None, 18432)             0         
                                                                 
 dense_8 (Dense)             (None, 256)               4718848   
                                                                 
 activation_478 (Activation)  (None, 256)              0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
 activation_479 (Activation)  (None, 1)               

In [ ]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate = 1e-5)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train // batch_size,
                    epochs = epochs,
                    validation_data  = val_generator,
                    validation_steps = nb_val // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


35/35 [==============================] - 45s 1s/step - loss: 0.5141 - accuracy: 0.8043 - val_loss: 0.0267 - val_accuracy: 0.9929


In [ ]:
scores = model.evaluate_generator(test_generator, nb_test // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
print(scores)

[0.03434102237224579, 0.9857142567634583]
